In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter

### ICO (*International Coffee Organization*) website data preprocessing in Coffee production category

In [16]:
# List of excel data available in from ICO website (https://www.ico.org/new_historical.asp)

url_list = ["https://www.ico.org/historical/1990%20onwards/Excel/1a%20-%20Total%20production.xlsx",
            "https://www.ico.org/historical/1990%20onwards/Excel/1b%20-%20Domestic%20consumption.xlsx",
            "https://www.ico.org/historical/1990%20onwards/Excel/2a%20-%20Exports%20-%20calendar%20year.xlsx",
            "https://www.ico.org/historical/1990%20onwards/Excel/2b%20-%20Imports.xlsx",
            "https://www.ico.org/historical/1990%20onwards/Excel/2c%20-%20Re-exports.xlsx",
            "https://www.ico.org/historical/1990%20onwards/Excel/4a%20-%20Inventories.xlsx",
            "https://www.ico.org/historical/1990%20onwards/Excel/4b%20-%20Disappearance.xlsx"]

In [22]:
# Function for data extraction and save each excel file as dataframe

df_list = []

def data_loading(url):
    """
    Function for data extraction from website excel file.
    
    Parameter:
        - url: list of url available from ICO website
    Returns:
        Separate DataFrame with data extracted from url list
    """
    
    for i in url:
        df = pd.read_excel(i, header=3)
        df_list.append(df)
        
data_loading(url_list)

# Pack each of excel data into dataframe

df_production = df_list[0]
df_domestic_consumption = df_list[1]
df_export = df_list[2]
df_import = df_list[3]
df_re_export = df_list[4]
df_inventorie = df_list[5]
df_import_consumption = df_list[6]

C:\Users\dell\miniconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\dell\miniconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [23]:
# Function for data preprocessing (cleaning, value change, deduplication, droping rows and values)

def data_production_preprocesing(*args):
    """
    Function for data preprocesing for df_production
    
    Returns:
        Clean version of df_production DataFrame
    """
    global df_production
    
    # Data preprocesing, variable rename
    df_production = df_production.rename(columns={"Unnamed: 1" : "Coffee type", "Crop year" : "Country"})
    df_production["Coffee type"].value_counts()
    df_production["Coffee type"].replace({"(A)" : "Arabica", "(R)" : "Robusta", "(R/A)" : "Robusta/Arabica",
                           "(A/R)" : "Arabica/Robusta", 0 : "Arabica"}, inplace=True)
    df_production.dropna(how='all', inplace=True)
    df_production['Coffee type'].isna()
    df_production.drop(index=[1, 17, 26, 62, 63], axis=0, inplace=True)
    df_production.reset_index(drop=True)
    
    # Rounding numeric values
    df_production_1 = df_production.iloc[:, 2:32]
    df_production_1.reset_index(drop=True)
    df_production_2 = df_production.iloc[:, 0:2]
    df_production_2.reset_index(drop=True)
    
    df_production_1 = df_production_1.apply(lambda x : round(x, 0))
    
    # In order to obtain kg values, data have to be multiplaying by 60 kg coffee bags  
    df_production_1 = df_production_1.apply(lambda x : x * 60)
    
    # Data type change from float to intiger
    df_production_1[[col for col in df_production_1.columns if df_production_1[col].dtypes == float]] = \
    df_production_1[[col for col in df_production_1.columns if df_production_1[col].dtypes == float]].astype(int)
    df_production = pd.concat([df_production_2, df_production_1], axis=1).reset_index(drop=True)
    
    # Total consumption
    df_production['Total_production'] = df_production.iloc[:, 2:33].sum(axis=1)
    
    return df_production

df_production = data_production_preprocesing()

In [25]:
df_production.head(5)

,Country,Coffee type,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,...,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20,Total_production
0,Angola,Robusta/Arabica,3000,4740,4680,1980,4620,3720,4260,3840,...,1740,1980,2100,2340,2460,2700,2100,2520,3120,82080
1,Bolivia (Plurinational State of),Arabica,7380,6240,7200,3060,7020,8520,7500,8460,...,7920,6300,7200,6000,5040,4680,5040,4980,4860,207000
2,Brazil,Arabica/Robusta,1637160,1637580,2076180,1690020,1691520,1083600,1751820,1568880,...,2915520,3325080,3281340,3198300,3172260,3407280,3164400,3907860,3492660,75082980
3,Burundi,Arabica/Robusta,29220,40020,37200,23580,39840,26040,24060,15000,...,12240,24360,9780,14880,16140,11760,12120,12240,16320,623640
4,Ecuador,Arabica/Robusta,90240,127440,71100,124140,142560,113280,119580,71460,...,49500,49680,39960,38640,38640,38700,37440,29760,33540,1900380


In [27]:
def data_domestic_consumption_preprocesing(*args):
    """
    Function for data preprocesing a df_domestic_consumption
    
    Returns:
        Clean version of df_domestic_consumption DataFrame
    """
    global df_domestic_consumption
    
    # Data preprocesing, variable rename
    df_domestic_consumption = df_domestic_consumption.rename(columns={"Unnamed: 1" : "Coffee type", "Crop year" : "Country"})
    df_domestic_consumption["Coffee type"].value_counts()
    df_domestic_consumption["Coffee type"].replace({"(A)" : "Arabica", "(R)" : "Robusta", "(R/A)" : "Robusta/Arabica",
                           "(A/R)" : "Arabica/Robusta"}, inplace=True)
    df_domestic_consumption["Coffee type"].value_counts()
    df_domestic_consumption.dropna(how='all', inplace=True)
    df_domestic_consumption['Coffee type'].isna()
    df_domestic_consumption.drop(index=[1, 17, 26, 62, 63], axis=0, inplace=True)
    df_domestic_consumption.reset_index(drop=True)
    
    # Rounding numeric values
    df_domestic_consumption_1 = df_domestic_consumption.iloc[:, 2:32]
    df_domestic_consumption_1.reset_index(drop=True)
    df_domestic_consumption_2 = df_domestic_consumption.iloc[:, 0:2]
    df_domestic_consumption_2.reset_index(drop=True)
    
    # In order to obtain kg values, data have to be multiplaying by 60 kg coffee bags
    df_domestic_consumption_1 = df_domestic_consumption_1.apply(lambda x : x * 60)
    
    # Data type change from float to intiger
    df_domestic_consumption_1[[col for col in df_domestic_consumption_1.columns if df_domestic_consumption_1[col].dtypes == float]] = \
    df_domestic_consumption_1[[col for col in df_domestic_consumption_1.columns if df_domestic_consumption_1[col].dtypes == float]].astype(int)
    df_domestic_consumption = pd.concat([df_domestic_consumption_2, df_domestic_consumption_1], axis=1).reset_index(drop=True)
    
    # Total consumption
    df_domestic_consumption['Total_domestic_consumption'] = df_domestic_consumption.iloc[:, 2:33].sum(axis=1)
    
    return df_domestic_consumption

df_domestic_consumption = data_domestic_consumption_preprocesing()

In [28]:
df_domestic_consumption.head(5)

,Country,Coffee type,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,...,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20,Total_domestic_consumption
0,Angola,Robusta/Arabica,1200,1800,2100,1200,1500,600,1200,2400,...,1800,1800,1800,1800,1800,1800,1800,1800,1800,46500
1,Bolivia (Plurinational State of),Arabica,1500,1620,1650,1710,1770,1830,1890,1950,...,2940,3030,3120,3210,3300,3420,3510,3600,3660,75180
2,Brazil,Arabica/Robusta,492000,510000,534000,546000,558000,606000,660000,690000,...,1183200,1219800,1205100,1219980,1230480,1273500,1319820,1332000,1320000,27824700
3,Burundi,Arabica/Robusta,120,96,102,114,120,120,120,120,...,120,120,120,120,120,120,120,120,120,3409
4,Ecuador,Arabica/Robusta,21000,21000,21000,21000,21000,21000,18000,18000,...,9000,9000,9300,9300,9300,9300,9300,9300,8940,381540


In [36]:
path = r"C:\Users\dell\Desktop\Coffee_data.xlsx"

writer = pd.ExcelWriter(path, engine= "xlsxwriter")
df_production.to_excel(writer, sheet_name = "Production", index=False)
df_domestic_consumption .to_excel(writer, sheet_name = "Domestic consumption", index=False)

writer.save()
writer.close()

C:\Users\dell\miniconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
